<div style="
    background-color: #E3F2FD; 
    padding: 20px; 
    text-align: center;"
    >
    <h1 style="color: darkblue; font-family: Poppins, sans-serif; margin-bottom: 5px; font-weight: bold;">
        Triển khai mô hình học máy/ học sâu trong việc dự đoán dịch Covid-19 trong tương lai
    </h1>
    <h3 style="color:darkblue; font-family: Poppins, sans-serif; margin-top: 0;">
        Nhóm 9
    </h3>
<hr style="border: 2x solid darkblue;">
</div>


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from models.prophet_lstm_ensemble import ProphetLSTMEnsemble

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
sns.set_palette("Set2")

MODEL_DIR = "models"
DATA_DIR = "../data/processed/daily_data"

CONFIRMED_FILE = os.path.join(DATA_DIR, "daily_confirmed_cases.csv")
DEATHS_FILE = os.path.join(DATA_DIR, "daily_deaths_cases.csv")
RECOVERED_FILE = os.path.join(DATA_DIR, "daily_recovered_cases.csv")

country = "Vietnam"
test_size = 30
future_periods = 60
prophet_weight = 0.6
lstm_weight = 0.4
stacking = True
lstm_epochs = 50
force_retrain = False
save_model = True

os.makedirs("results", exist_ok=True)

### 1. Mô hình dự đoán số ca nhiễm
Khởi tạo và huấn luyện mô hình ensemble cho dữ liệu số ca nhiễm (Confirmed). Tải và hiển thị dữ liệu, chia tập train/test, kiểm tra và huấn luyện mô hình (Prophet và LSTM). Đánh giá mô hình trên tập test bằng các chỉ số MAE, RMSE, MAPE. Vẽ biểu đồ so sánh kết quả dự đoán trên tập test và dự đoán 60 ngày tương lai. Cuối cùng, lưu kết quả dự đoán vào file CSV và hiển thị 5 ngày dự đoán đầu tiên.

In [ ]:
print(f"\n{'='*80}")
print(f"  HUẤN LUYỆN MÔ HÌNH CHO CONFIRMED - {country}")
print(f"{'='*80}")

confirmed_model = ProphetLSTMEnsemble(
    ensemble_method='weighted_average',
    prophet_weight=prophet_weight,
    lstm_weight=lstm_weight,
    stacking=stacking
)

confirmed_data = confirmed_model.load_and_prepare_data(CONFIRMED_FILE, country, "Confirmed")
confirmed_train, confirmed_test = confirmed_model.split_train_test(confirmed_data, test_size)

print(f"Dữ liệu Confirmed cho {country}:")
print(f"- Tổng số ngày: {len(confirmed_data)}")
print(f"- Thời gian: từ {confirmed_data['Date'].min().date()} đến {confirmed_data['Date'].max().date()}")
print(f"- Tập train: {len(confirmed_train)} ngày")
print(f"- Tập test: {len(confirmed_test)} ngày")

plt.figure(figsize=(14, 6))
plt.plot(confirmed_data['Date'], confirmed_data["Confirmed"])
plt.title(f'Dữ liệu số ca nhiễm cho {country}')
plt.xlabel('Ngày')
plt.ylabel('Số ca nhiễm')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
confirmed_model_exists = confirmed_model.check_model_exists(country, "Confirmed")

if confirmed_model_exists and not force_retrain:
    print(f"Mô hình cho {country} - Confirmed đã tồn tại. Đang tải mô hình...")
    confirmed_model.load_models(country, "Confirmed")
    print("Đã tải mô hình thành công!")
else:
    if confirmed_model_exists and force_retrain:
        print(f"Bắt buộc huấn luyện lại mô hình Confirmed...")
    else:
        print(f"Huấn luyện mô hình mới cho Confirmed...")
    
    confirmed_prophet_data = confirmed_model.prepare_prophet_data(confirmed_train, "Confirmed")
    
    print("Đang huấn luyện mô hình Prophet...")
    confirmed_prophet_forecast = confirmed_model.fit_prophet(confirmed_prophet_data, test_size, future_periods)
    
    confirmed_prophet_train_pred = confirmed_prophet_forecast.iloc[:len(confirmed_train)]['yhat'].values
    
    print("Đang chuẩn bị dữ liệu cho LSTM...")
    if stacking:
        confirmed_X_train, confirmed_y_train, _ = confirmed_model.prepare_stacking_data(
            confirmed_prophet_train_pred, 
            confirmed_train["Confirmed"].values, 
            confirmed_model.look_back
        )
    else:
        confirmed_X_train, confirmed_y_train, _ = confirmed_model.prepare_lstm_data(
            confirmed_train["Confirmed"].values,
            confirmed_model.look_back
        )
    
    print(f"Đang huấn luyện LSTM với {lstm_epochs} epochs...")
    confirmed_history = confirmed_model.fit_lstm(confirmed_X_train, confirmed_y_train, epochs=lstm_epochs)
    
    if save_model:
        print("Đang lưu mô hình...")
        confirmed_model_dir = confirmed_model.save_models(country, "Confirmed")
        print(f"Đã lưu mô hình vào: {confirmed_model_dir}")

In [ ]:
print("\nĐánh giá mô hình trên tập test:")

confirmed_prophet_test_pred = confirmed_model.prophet_forecast.iloc[-test_size-future_periods:-future_periods]['yhat'].values

if confirmed_model.stacking:
    confirmed_prophet_train_pred = confirmed_model.prophet_forecast.iloc[:len(confirmed_train)]['yhat'].values
    
    confirmed_train_combined = np.column_stack([
        confirmed_train["Confirmed"].values, 
        confirmed_prophet_train_pred
    ])
    
    confirmed_train_scaled = confirmed_model.scaler.transform(confirmed_train_combined)
    
    confirmed_X_train = []
    confirmed_y_train = []
    look_back = confirmed_model.look_back
    
    for i in range(len(confirmed_train_scaled) - look_back):
        confirmed_X_train.append(confirmed_train_scaled[i:i+look_back])
        confirmed_y_train.append(confirmed_train_scaled[i+look_back, 0])
    
    confirmed_X_train = np.array(confirmed_X_train)
    confirmed_y_train = np.array(confirmed_y_train)
    
    confirmed_lstm_test_pred = []
    confirmed_last_sequence = confirmed_X_train[-1].reshape(1, look_back, 2)
    
    for _ in range(test_size):
        next_pred = confirmed_model.lstm_model.predict(confirmed_last_sequence)
        confirmed_lstm_test_pred.append(next_pred[0, 0])
        
        confirmed_last_sequence = np.roll(confirmed_last_sequence, -1, axis=1)
        confirmed_last_sequence[0, -1, 0] = next_pred
        confirmed_last_sequence[0, -1, 1] = confirmed_prophet_test_pred[len(confirmed_lstm_test_pred)-1]
    
    confirmed_lstm_test_pred = confirmed_model.scaler.inverse_transform(
        np.column_stack([
            np.array(confirmed_lstm_test_pred), 
            confirmed_prophet_test_pred[:len(confirmed_lstm_test_pred)]
        ])
    )[:, 0]

else:
    confirmed_X_train, confirmed_y_train, confirmed_scaled_train = confirmed_model.prepare_lstm_data(
        confirmed_train["Confirmed"].values,
        confirmed_model.look_back
    )
    
    confirmed_lstm_test_pred = []
    confirmed_last_sequence = confirmed_scaled_train[-confirmed_model.look_back:].reshape(1, confirmed_model.look_back, 1)
    
    for _ in range(test_size):
        next_pred = confirmed_model.lstm_model.predict(confirmed_last_sequence)
        confirmed_lstm_test_pred.append(next_pred[0, 0])
        confirmed_last_sequence = np.roll(confirmed_last_sequence, -1, axis=1)
        confirmed_last_sequence[0, -1, 0] = next_pred
    
    confirmed_lstm_test_pred = confirmed_model.scaler.inverse_transform(
        np.array(confirmed_lstm_test_pred).reshape(-1, 1)
    ).flatten()

confirmed_ensemble_test_pred = (
    prophet_weight * confirmed_prophet_test_pred +
    lstm_weight * confirmed_lstm_test_pred
)

confirmed_metrics = confirmed_model.evaluate(
    confirmed_test, 
    "Confirmed", 
    confirmed_prophet_test_pred, 
    confirmed_lstm_test_pred, 
    confirmed_ensemble_test_pred
)

for model_name, model_metrics in confirmed_metrics.items():
    print(f"{model_name}:")
    for metric_name, metric_value in model_metrics.items():
        print(f"  {metric_name}: {metric_value:.2f}")

In [ ]:
print(f"\nDự đoán {future_periods} ngày trong tương lai:")
confirmed_future = confirmed_model.predict(confirmed_data, "Confirmed", future_periods)

# Ensure consistent length
min_length = min(
    len(confirmed_test['Date'].values),
    len(confirmed_test["Confirmed"].values),
    len(confirmed_prophet_test_pred),
    len(confirmed_lstm_test_pred),
    len(confirmed_ensemble_test_pred)
)

confirmed_test_predictions = pd.DataFrame({
    'Date': confirmed_test['Date'].values[:min_length],
    'Actual': confirmed_test["Confirmed"].values[:min_length],
    'Prophet': confirmed_prophet_test_pred[:min_length],
    'LSTM': confirmed_lstm_test_pred[:min_length],
    'Ensemble': confirmed_ensemble_test_pred[:min_length]
})

# Debug print to check lengths
print("Lengths:")
print("Date:", len(confirmed_test_predictions['Date']))
print("Actual:", len(confirmed_test_predictions['Actual']))
print("Prophet:", len(confirmed_test_predictions['Prophet']))
print("LSTM:", len(confirmed_test_predictions['LSTM']))
print("Ensemble:", len(confirmed_test_predictions['Ensemble']))

plt.figure(figsize=(14, 6))
plt.plot(confirmed_test_predictions['Date'], confirmed_test_predictions['Actual'], label='Thực tế', linewidth=2)
plt.plot(confirmed_test_predictions['Date'], confirmed_test_predictions['Prophet'], label='Prophet', linestyle='--')
plt.plot(confirmed_test_predictions['Date'], confirmed_test_predictions['LSTM'], label='LSTM', linestyle='--')
plt.plot(confirmed_test_predictions['Date'], confirmed_test_predictions['Ensemble'], label='Ensemble', linewidth=2)

plt.title(f'So sánh kết quả dự đoán trên tập test - Số ca nhiễm cho {country}')
plt.xlabel('Ngày')
plt.ylabel('Số ca nhiễm')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(confirmed_data['Date'], confirmed_data["Confirmed"], label='Dữ liệu lịch sử', color='blue')
plt.plot(confirmed_future['Date'], confirmed_future['Prophet_Prediction'], label='Prophet', linestyle='--', color='red')
plt.plot(confirmed_future['Date'], confirmed_future['LSTM_Prediction'], label='LSTM', linestyle='--', color='orange')
plt.plot(confirmed_future['Date'], confirmed_future['Ensemble_Prediction'], label='Ensemble', linewidth=2, color='purple')

plt.fill_between(
    confirmed_future['Date'],
    confirmed_future['Lower_CI'],
    confirmed_future['Upper_CI'],
    alpha=0.2, color='red', label='Khoảng tin cậy 95%'
)

plt.title(f'Dự đoán số ca nhiễm cho {country} trong {future_periods} ngày tới')
plt.xlabel('Ngày')
plt.ylabel('Số ca nhiễm')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
confirmed_file = f"results/{country}_Confirmed_forecast_{timestamp}.csv"
confirmed_future.to_csv(confirmed_file, index=False)
print(f"Đã lưu kết quả dự đoán vào file: {confirmed_file}")

In [ ]:
confirmed_display = confirmed_future.copy()
confirmed_display['Date'] = confirmed_display['Date'].dt.strftime('%Y-%m-%d')
confirmed_display = confirmed_display.round(2)
print("\nDự đoán 5 ngày đầu tiên:")
print(confirmed_display.head(5))

### 2. Mô hình dự đoán số ca tử vong
Tương tự như phần số ca nhiễm, nhưng áp dụng cho dữ liệu số ca tử vong (Deaths). Tải và phân tích dữ liệu, huấn luyện mô hình, đánh giá trên tập test, dự đoán tương lai, và lưu kết quả.

In [ ]:
print(f"\n{'='*80}")
print(f"  HUẤN LUYỆN MÔ HÌNH CHO DEATHS - {country}")
print(f"{'='*80}")

deaths_model = ProphetLSTMEnsemble(
    ensemble_method='weighted_average',
    prophet_weight=prophet_weight,
    lstm_weight=lstm_weight,
    stacking=stacking
)

deaths_data = deaths_model.load_and_prepare_data(DEATHS_FILE, country, "Deaths")
deaths_train, deaths_test = deaths_model.split_train_test(deaths_data, test_size)

print(f"Dữ liệu Deaths cho {country}:")
print(f"- Tổng số ngày: {len(deaths_data)}")
print(f"- Thời gian: từ {deaths_data['Date'].min().date()} đến {deaths_data['Date'].max().date()}")
print(f"- Tập train: {len(deaths_train)} ngày")
print(f"- Tập test: {len(deaths_test)} ngày")

plt.figure(figsize=(14, 6))
plt.plot(deaths_data['Date'], deaths_data["Deaths"])
plt.title(f'Dữ liệu số ca tử vong cho {country}')
plt.xlabel('Ngày')
plt.ylabel('Số ca tử vong')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
deaths_model_exists = deaths_model.check_model_exists(country, "Deaths")

if deaths_model_exists and not force_retrain:
    print(f"Mô hình cho {country} - Deaths đã tồn tại. Đang tải mô hình...")
    deaths_model.load_models(country, "Deaths")
    print("Đã tải mô hình thành công!")
else:
    if deaths_model_exists and force_retrain:
        print(f"Bắt buộc huấn luyện lại mô hình Deaths...")
    else:
        print(f"Huấn luyện mô hình mới cho Deaths...")
    
    deaths_prophet_data = deaths_model.prepare_prophet_data(deaths_train, "Deaths")
    
    print("Đang huấn luyện mô hình Prophet...")
    deaths_prophet_forecast = deaths_model.fit_prophet(deaths_prophet_data, test_size, future_periods)
    
    deaths_prophet_train_pred = deaths_prophet_forecast.iloc[:len(deaths_train)]['yhat'].values
    
    print("Đang chuẩn bị dữ liệu cho LSTM...")
    if stacking:
        deaths_X_train, deaths_y_train, _ = deaths_model.prepare_stacking_data(
            deaths_prophet_train_pred, 
            deaths_train["Deaths"].values, 
            deaths_model.look_back
        )
    else:
        deaths_X_train, deaths_y_train, _ = deaths_model.prepare_lstm_data(
            deaths_train["Deaths"].values,
            deaths_model.look_back
        )
    
    print(f"Đang huấn luyện LSTM với {lstm_epochs} epochs...")
    deaths_history = deaths_model.fit_lstm(deaths_X_train, deaths_y_train, epochs=lstm_epochs)
    
    if save_model:
        print("Đang lưu mô hình...")
        deaths_model_dir = deaths_model.save_models(country, "Deaths")
        print(f"Đã lưu mô hình vào: {deaths_model_dir}")

In [ ]:
print("\nĐánh giá mô hình trên tập test:")

deaths_prophet_test_pred = deaths_model.prophet_forecast.iloc[-test_size-future_periods:-future_periods]['yhat'].values[:test_size]

if deaths_model.stacking:
    deaths_prophet_train_pred = deaths_model.prophet_forecast.iloc[:len(deaths_train)]['yhat'].values
    
    deaths_last_sequence = np.column_stack([
        deaths_train["Deaths"].values[-deaths_model.look_back:],
        deaths_prophet_train_pred[-deaths_model.look_back:]
    ])
    
    deaths_current_input = deaths_model.scaler.transform(deaths_last_sequence).reshape(1, deaths_model.look_back, 2)
    
    deaths_lstm_test_pred = []
    
    for _ in range(test_size):
        next_pred = deaths_model.lstm_model.predict(deaths_current_input)
        deaths_lstm_test_pred.append(next_pred[0, 0])
        
        deaths_current_input = np.roll(deaths_current_input, -1, axis=1)
        deaths_current_input[0, -1, 0] = next_pred
        
        if len(deaths_lstm_test_pred) <= len(deaths_prophet_test_pred):
            deaths_current_input[0, -1, 1] = deaths_prophet_test_pred[len(deaths_lstm_test_pred)-1]
        else:
            deaths_current_input[0, -1, 1] = next_pred
    
    deaths_lstm_test_pred = deaths_model.scaler.inverse_transform(
        np.column_stack([
            np.array(deaths_lstm_test_pred), 
            deaths_prophet_test_pred[:len(deaths_lstm_test_pred)]
        ])
    )[:, 0]

else:
    deaths_X_train, deaths_y_train, deaths_scaled_train = deaths_model.prepare_lstm_data(
        deaths_train["Deaths"].values,
        deaths_model.look_back
    )
    
    deaths_lstm_test_pred = []
    deaths_last_sequence = deaths_scaled_train[-deaths_model.look_back:].reshape(1, deaths_model.look_back, 1)
    
    for _ in range(test_size):
        next_pred = deaths_model.lstm_model.predict(deaths_last_sequence)
        deaths_lstm_test_pred.append(next_pred[0, 0])
        deaths_last_sequence = np.roll(deaths_last_sequence, -1, axis=1)
        deaths_last_sequence[0, -1, 0] = next_pred
    
    deaths_lstm_test_pred = deaths_model.scaler.inverse_transform(
        np.array(deaths_lstm_test_pred).reshape(-1, 1)
    ).flatten()

min_length = min(len(deaths_prophet_test_pred), len(deaths_lstm_test_pred))

deaths_prophet_test_pred = deaths_prophet_test_pred[:min_length]
deaths_lstm_test_pred = deaths_lstm_test_pred[:min_length]

deaths_ensemble_test_pred = (
    prophet_weight * deaths_prophet_test_pred +
    lstm_weight * deaths_lstm_test_pred
)

deaths_metrics = deaths_model.evaluate(
    deaths_test.iloc[:min_length], 
    "Deaths", 
    deaths_prophet_test_pred, 
    deaths_lstm_test_pred, 
    deaths_ensemble_test_pred
)

for model_name, model_metrics in deaths_metrics.items():
    print(f"{model_name}:")
    for metric_name, metric_value in model_metrics.items():
        print(f"  {metric_name}: {metric_value:.2f}")

print(f"\nDự đoán {future_periods} ngày trong tương lai:")
deaths_future = deaths_model.predict(deaths_data, "Deaths", future_periods)

In [ ]:
min_length = min(
    len(deaths_test['Date'].values),
    len(deaths_test["Deaths"].values),
    len(deaths_prophet_test_pred),
    len(deaths_lstm_test_pred),
    len(deaths_ensemble_test_pred)
)

deaths_test_predictions = pd.DataFrame({
    'Date': deaths_test['Date'].values[:min_length],
    'Actual': deaths_test["Deaths"].values[:min_length],
    'Prophet': deaths_prophet_test_pred[:min_length],
    'LSTM': deaths_lstm_test_pred[:min_length],
    'Ensemble': deaths_ensemble_test_pred[:min_length]
})

print("Lengths of arrays:")
print("Date:", len(deaths_test_predictions['Date']))
print("Actual:", len(deaths_test_predictions['Actual']))
print("Prophet:", len(deaths_test_predictions['Prophet']))
print("LSTM:", len(deaths_test_predictions['LSTM']))
print("Ensemble:", len(deaths_test_predictions['Ensemble']))

plt.figure(figsize=(14, 6))
plt.plot(deaths_test_predictions['Date'], deaths_test_predictions['Actual'], label='Thực tế', linewidth=2)
plt.plot(deaths_test_predictions['Date'], deaths_test_predictions['Prophet'], label='Prophet', linestyle='--')
plt.plot(deaths_test_predictions['Date'], deaths_test_predictions['LSTM'], label='LSTM', linestyle='--')
plt.plot(deaths_test_predictions['Date'], deaths_test_predictions['Ensemble'], label='Ensemble', linewidth=2)

plt.title(f'So sánh kết quả dự đoán trên tập test - Số ca tử vong cho {country}')
plt.xlabel('Ngày')
plt.ylabel('Số ca tử vong')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(deaths_data['Date'], deaths_data["Deaths"], label='Dữ liệu lịch sử', color='blue')
plt.plot(deaths_future['Date'], deaths_future['Prophet_Prediction'], label='Prophet', linestyle='--', color='red')
plt.plot(deaths_future['Date'], deaths_future['LSTM_Prediction'], label='LSTM', linestyle='--', color='orange')
plt.plot(deaths_future['Date'], deaths_future['Ensemble_Prediction'], label='Ensemble', linewidth=2, color='purple')

plt.fill_between(
    deaths_future['Date'],
    deaths_future['Lower_CI'],
    deaths_future['Upper_CI'],
    alpha=0.2, color='red', label='Khoảng tin cậy 95%'
)

plt.title(f'Dự đoán số ca tử vong cho {country} trong {future_periods} ngày tới')
plt.xlabel('Ngày')
plt.ylabel('Số ca tử vong')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
deaths_file = f"results/{country}_Deaths_forecast_{timestamp}.csv"
deaths_future.to_csv(deaths_file, index=False)
print(f"Đã lưu kết quả dự đoán vào file: {deaths_file}")

deaths_display = deaths_future.copy()
deaths_display['Date'] = deaths_display['Date'].dt.strftime('%Y-%m-%d')
deaths_display = deaths_display.round(2)
print("\nDự đoán 5 ngày đầu tiên:")
print(deaths_display.head(5))

### 3. Mô hình dự đoán số ca hồi phục
Tương tự như hai phần trên, nhưng áp dụng cho dữ liệu số ca hồi phục (Recovered). Thực hiện các bước tải dữ liệu, huấn luyện, đánh giá và dự đoán tương tự.

In [ ]:
print(f"\n{'='*80}")
print(f"  HUẤN LUYỆN MÔ HÌNH CHO RECOVERED - {country}")
print(f"{'='*80}")

recovered_model = ProphetLSTMEnsemble(
    ensemble_method='weighted_average',
    prophet_weight=prophet_weight,
    lstm_weight=lstm_weight,
    stacking=stacking
)

recovered_data = recovered_model.load_and_prepare_data(RECOVERED_FILE, country, "Recovered")
recovered_train, recovered_test = recovered_model.split_train_test(recovered_data, test_size)

print(f"Dữ liệu Recovered cho {country}:")
print(f"- Tổng số ngày: {len(recovered_data)}")
print(f"- Thời gian: từ {recovered_data['Date'].min().date()} đến {recovered_data['Date'].max().date()}")
print(f"- Tập train: {len(recovered_train)} ngày")
print(f"- Tập test: {len(recovered_test)} ngày")

plt.figure(figsize=(14, 6))
plt.plot(recovered_data['Date'], recovered_data["Recovered"])
plt.title(f'Dữ liệu số ca hồi phục cho {country}')
plt.xlabel('Ngày')
plt.ylabel('Số ca hồi phục')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

recovered_model_exists = recovered_model.check_model_exists(country, "Recovered")

if recovered_model_exists and not force_retrain:
    print(f"Mô hình cho {country} - Recovered đã tồn tại. Đang tải mô hình...")
    recovered_model.load_models(country, "Recovered")
    print("Đã tải mô hình thành công!")
else:
    if recovered_model_exists and force_retrain:
        print(f"Bắt buộc huấn luyện lại mô hình Recovered...")
    else:
        print(f"Huấn luyện mô hình mới cho Recovered...")
    
    recovered_prophet_data = recovered_model.prepare_prophet_data(recovered_train, "Recovered")
    
    print("Đang huấn luyện mô hình Prophet...")
    recovered_prophet_forecast = recovered_model.fit_prophet(recovered_prophet_data, test_size, future_periods)
    
    recovered_prophet_train_pred = recovered_prophet_forecast.iloc[:len(recovered_train)]['yhat'].values
    
    print("Đang chuẩn bị dữ liệu cho LSTM...")
    if stacking:
        recovered_X_train, recovered_y_train, _ = recovered_model.prepare_stacking_data(
            recovered_prophet_train_pred, 
            recovered_train["Recovered"].values, 
            recovered_model.look_back
        )
    else:
        recovered_X_train, recovered_y_train, _ = recovered_model.prepare_lstm_data(
            recovered_train["Recovered"].values,
            recovered_model.look_back
        )
    
    print(f"Đang huấn luyện LSTM với {lstm_epochs} epochs...")
    recovered_history = recovered_model.fit_lstm(recovered_X_train, recovered_y_train, epochs=lstm_epochs)
    
    if save_model:
        print("Đang lưu mô hình...")
        recovered_model_dir = recovered_model.save_models(country, "Recovered")
        print(f"Đã lưu mô hình vào: {recovered_model_dir}")

In [ ]:
print("\nĐánh giá mô hình trên tập test:")

recovered_prophet_test_pred = recovered_model.prophet_forecast.iloc[-test_size-future_periods:-future_periods]['yhat'].values

if recovered_model.stacking:
    recovered_prophet_train_pred = recovered_model.prophet_forecast.iloc[:len(recovered_train)]['yhat'].values
    
    recovered_last_sequence = np.column_stack([
        recovered_train["Recovered"].values[-recovered_model.look_back:],
        recovered_prophet_train_pred[-recovered_model.look_back:]
    ])
    
    recovered_current_input = recovered_model.scaler.transform(recovered_last_sequence).reshape(1, recovered_model.look_back, 2)
    
    recovered_lstm_test_pred = []
    
    for _ in range(test_size):
        next_pred = recovered_model.lstm_model.predict(recovered_current_input)
        recovered_lstm_test_pred.append(next_pred[0, 0])
        
        recovered_current_input = np.roll(recovered_current_input, -1, axis=1)
        recovered_current_input[0, -1, 0] = next_pred
        
        if len(recovered_lstm_test_pred) <= len(recovered_prophet_test_pred):
            recovered_current_input[0, -1, 1] = recovered_prophet_test_pred[len(recovered_lstm_test_pred)-1]
        else:
            recovered_current_input[0, -1, 1] = next_pred
    
    recovered_lstm_test_pred = recovered_model.scaler.inverse_transform(
        np.column_stack([
            np.array(recovered_lstm_test_pred), 
            recovered_prophet_test_pred[:len(recovered_lstm_test_pred)]
        ])
    )[:, 0]

else:
    recovered_X_train, recovered_y_train, recovered_scaled_train = recovered_model.prepare_lstm_data(
        recovered_train["Recovered"].values,
        recovered_model.look_back
    )
    
    recovered_lstm_test_pred = []
    recovered_last_sequence = recovered_scaled_train[-recovered_model.look_back:].reshape(1, recovered_model.look_back, 1)
    
    for _ in range(test_size):
        next_pred = recovered_model.lstm_model.predict(recovered_last_sequence)
        recovered_lstm_test_pred.append(next_pred[0, 0])
        recovered_last_sequence = np.roll(recovered_last_sequence, -1, axis=1)
        recovered_last_sequence[0, -1, 0] = next_pred
    
    recovered_lstm_test_pred = recovered_model.scaler.inverse_transform(
        np.array(recovered_lstm_test_pred).reshape(-1, 1)
    ).flatten()

min_length = min(len(recovered_prophet_test_pred), len(recovered_lstm_test_pred))

recovered_prophet_test_pred = recovered_prophet_test_pred[:min_length]
recovered_lstm_test_pred = recovered_lstm_test_pred[:min_length]

recovered_ensemble_test_pred = (
    prophet_weight * recovered_prophet_test_pred +
    lstm_weight * recovered_lstm_test_pred
)

recovered_metrics = recovered_model.evaluate(
    recovered_test.iloc[:min_length], 
    "Recovered", 
    recovered_prophet_test_pred, 
    recovered_lstm_test_pred, 
    recovered_ensemble_test_pred
)

for model_name, model_metrics in recovered_metrics.items():
    print(f"{model_name}:")
    for metric_name, metric_value in model_metrics.items():
        print(f"  {metric_name}: {metric_value:.2f}")

In [ ]:
print(f"\nDự đoán {future_periods} ngày trong tương lai:")
recovered_future = recovered_model.predict(recovered_data, "Recovered", future_periods)

min_length = min(
    len(recovered_test['Date'].values),
    len(recovered_test["Recovered"].values),
    len(recovered_prophet_test_pred),
    len(recovered_lstm_test_pred),
    len(recovered_ensemble_test_pred)
)

recovered_test_predictions = pd.DataFrame({
    'Date': recovered_test['Date'].values[:min_length],
    'Actual': recovered_test["Recovered"].values[:min_length],
    'Prophet': recovered_prophet_test_pred[:min_length],
    'LSTM': recovered_lstm_test_pred[:min_length],
    'Ensemble': recovered_ensemble_test_pred[:min_length]
})

print("Lengths of arrays:")
print("Date:", len(recovered_test_predictions['Date']))
print("Actual:", len(recovered_test_predictions['Actual']))
print("Prophet:", len(recovered_test_predictions['Prophet']))
print("LSTM:", len(recovered_test_predictions['LSTM']))
print("Ensemble:", len(recovered_test_predictions['Ensemble']))

plt.figure(figsize=(14, 6))
plt.plot(recovered_test_predictions['Date'], recovered_test_predictions['Actual'], label='Thực tế', linewidth=2)
plt.plot(recovered_test_predictions['Date'], recovered_test_predictions['Prophet'], label='Prophet', linestyle='--')
plt.plot(recovered_test_predictions['Date'], recovered_test_predictions['LSTM'], label='LSTM', linestyle='--')
plt.plot(recovered_test_predictions['Date'], recovered_test_predictions['Ensemble'], label='Ensemble', linewidth=2)

plt.title(f'So sánh kết quả dự đoán trên tập test - Số ca hồi phục cho {country}')
plt.xlabel('Ngày')
plt.ylabel('Số ca hồi phục')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(recovered_data['Date'], recovered_data["Recovered"], label='Dữ liệu lịch sử', color='blue')
plt.plot(recovered_future['Date'], recovered_future['Prophet_Prediction'], label='Prophet', linestyle='--', color='red')
plt.plot(recovered_future['Date'], recovered_future['LSTM_Prediction'], label='LSTM', linestyle='--', color='orange')
plt.plot(recovered_future['Date'], recovered_future['Ensemble_Prediction'], label='Ensemble', linewidth=2, color='purple')

plt.fill_between(
    recovered_future['Date'],
    recovered_future['Lower_CI'],
    recovered_future['Upper_CI'],
    alpha=0.2, color='red', label='Khoảng tin cậy 95%'
)

plt.title(f'Dự đoán số ca hồi phục cho {country} trong {future_periods} ngày tới')
plt.xlabel('Ngày')
plt.ylabel('Số ca hồi phục')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
recovered_file = f"results/{country}_Recovered_forecast_{timestamp}.csv"
recovered_future.to_csv(recovered_file, index=False)
print(f"Đã lưu kết quả dự đoán vào file: {recovered_file}")

recovered_display = recovered_future.copy()
recovered_display['Date'] = recovered_display['Date'].dt.strftime('%Y-%m-%d')
recovered_display = recovered_display.round(2)
print("\nDự đoán 5 ngày đầu tiên:")
print(recovered_display.head(5))

### 4. So sánh và tổng hợp kết quả
Thu thập và gộp tất cả các metrics đánh giá từ ba loại dữ liệu, tạo DataFrame để so sánh. Vẽ các biểu đồ so sánh hiệu suất (MAE, RMSE, MAPE) giữa các mô hình cho từng loại dữ liệu. Cuối cùng, vẽ biểu đồ tổng hợp hiển thị dự đoán cho cả ba loại dữ liệu trên cùng một hình.

In [ ]:
print(f"\n{'='*80}")
print(f"  SO SÁNH HIỆU SUẤT CÁC MÔ HÌNH CHO {country}")
print(f"{'='*80}")

metrics_comparison = []

for model_name, model_metrics in confirmed_metrics.items():
    for metric_name, metric_value in model_metrics.items():
        metrics_comparison.append({
            'Data Type': 'Confirmed',
            'Model': model_name,
            'Metric': metric_name,
            'Value': metric_value
        })

for model_name, model_metrics in deaths_metrics.items():
    for metric_name, metric_value in model_metrics.items():
        metrics_comparison.append({
            'Data Type': 'Deaths',
            'Model': model_name,
            'Metric': metric_name,
            'Value': metric_value
        })

for model_name, model_metrics in recovered_metrics.items():
    for metric_name, metric_value in model_metrics.items():
        metrics_comparison.append({
            'Data Type': 'Recovered',
            'Model': model_name,
            'Metric': metric_name,
            'Value': metric_value
        })

metrics_df = pd.DataFrame(metrics_comparison)

In [ ]:

mae_df = metrics_df[metrics_df['Metric'] == 'MAE'].pivot(
    index='Model', columns='Data Type', values='Value'
)

plt.figure(figsize=(12, 6))
mae_df.plot(kind='bar')
plt.title('So sánh MAE giữa các mô hình')
plt.ylabel('MAE')
plt.xlabel('Mô hình')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:

rmse_df = metrics_df[metrics_df['Metric'] == 'RMSE'].pivot(
    index='Model', columns='Data Type', values='Value'
)

plt.figure(figsize=(12, 6))
rmse_df.plot(kind='bar')
plt.title('So sánh RMSE giữa các mô hình')
plt.ylabel('RMSE')
plt.xlabel('Mô hình')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:

mape_df = metrics_df[metrics_df['Metric'] == 'MAPE'].pivot(
    index='Model', columns='Data Type', values='Value'
)

plt.figure(figsize=(12, 6))
mape_df.plot(kind='bar')
plt.title('So sánh MAPE (%) giữa các mô hình')
plt.ylabel('MAPE (%)')
plt.xlabel('Mô hình')
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(15, 12))

plt.subplot(3, 1, 1)
plt.plot(confirmed_data['Date'], confirmed_data["Confirmed"], label='Dữ liệu lịch sử', color='blue')
plt.plot(confirmed_future['Date'], confirmed_future['Ensemble_Prediction'], 
         label='Dự đoán Confirmed', linewidth=2, color='red')
plt.fill_between(
    confirmed_future['Date'],
    confirmed_future['Lower_CI'],
    confirmed_future['Upper_CI'],
    alpha=0.2, color='red'
)
plt.title(f'Dự đoán số ca nhiễm cho {country}')
plt.xlabel('Ngày')
plt.ylabel('Số ca nhiễm')
plt.legend()
plt.grid(True)

plt.subplot(3, 1, 2)
plt.plot(deaths_data['Date'], deaths_data["Deaths"], label='Dữ liệu lịch sử', color='blue')
plt.plot(deaths_future['Date'], deaths_future['Ensemble_Prediction'], 
         label='Dự đoán Deaths', linewidth=2, color='purple')
plt.fill_between(
    deaths_future['Date'],
    deaths_future['Lower_CI'],
    deaths_future['Upper_CI'],
    alpha=0.2, color='purple'
)
plt.title(f'Dự đoán số ca tử vong cho {country}')
plt.xlabel('Ngày')
plt.ylabel('Số ca tử vong')
plt.legend()
plt.grid(True)

plt.subplot(3, 1, 3)
plt.plot(recovered_data['Date'], recovered_data["Recovered"], label='Dữ liệu lịch sử', color='blue')
plt.plot(recovered_future['Date'], recovered_future['Ensemble_Prediction'], 
         label='Dự đoán Recovered', linewidth=2, color='green')
plt.fill_between(
    recovered_future['Date'],
    recovered_future['Lower_CI'],
    recovered_future['Upper_CI'],
    alpha=0.2, color='green'
)
plt.title(f'Dự đoán số ca hồi phục cho {country}')
plt.xlabel('Ngày')
plt.ylabel('Số ca hồi phục')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"\nĐã hoàn thành huấn luyện và dự đoán cho tất cả loại dữ liệu COVID-19 của {country}!")